In [124]:
# loading required libraries
import pandas as pd
import matplotlib.pyplot as plt

In [126]:
# loading integrated dataset
data = pd.read_csv("pre_processed_data.csv")
data.head()

,area_code,area,year_code,year,item_code,item,producer_price,producer_price_index,area_harvested,production,...,total_pesticide_use_per_value_of_agri_production,total_pesticide_export_quantity,total_pesticide_export_value,total_pesticide_import_quantity,total_pesticide_import_value,temp_change_dec_jan_feb,temp_change_mar_apr_may,temp_change_jun_jul_aug,temp_change_sep_oct_nov,temp_change_meteorological_year
0,1,Armenia,1997,1997,15,Wheat,225.3,89.40,108000.0,183700.0,...,0.05,3.604,3.687,117.861,298.379,2.166,-0.319,0.870,0.184,0.725
1,1,Armenia,1998,1998,15,Wheat,205.6,83.91,118300.0,244300.0,...,0.06,3.604,3.687,207.171,340.124,-0.259,1.483,2.145,2.027,1.349
2,1,Armenia,1999,1999,15,Wheat,177.5,76.79,110129.0,214380.0,...,0.07,3.604,3.687,296.481,381.869,3.535,0.814,1.611,0.264,1.556
3,1,Armenia,2000,2000,15,Wheat,163.5,71.30,106581.0,177762.0,...,0.08,3.604,3.687,160.902,482.955,1.148,0.723,2.166,0.364,1.100
4,1,Armenia,2001,2001,15,Wheat,166.0,74.51,108554.0,241679.0,...,0.09,1.145,9.068,200.884,698.022,1.818,2.072,1.578,0.409,1.469


In [127]:
prop_na = (data.isna().mean()*100).sort_values(ascending=False)
prop_na = prop_na.loc[prop_na > 10]
prop_na

fdi_ag_forest_fish_share                      57.006191
fdi_ag_forest_fish                            57.006191
govt_expenditure_on_ag_forest_fish            47.790327
producer_price                                44.675760
agri_orientation_index_2015_usd               39.907767
credit_to_ag_forest_fish_share_totalcredit    39.907767
credit_to_ag_forest_fish_2015_usd             39.907767
afs_employment_share_in_total_employment      38.795271
total_employment_afs                          37.964333
total_govt_expenditure                        32.366458
agri_orientation_index_govt_expenditure       32.339517
area_temporary_crops                          32.057960
phosphorus_production                         27.906436
export_quantity                               26.466953
export_value                                  26.245087
gross_production_value                        26.147361
import_quantity                               25.503951
import_value                                  25

In [128]:
feature_list = list(prop_na.keys())
prop_list = list(round(val,3) for val in prop_na.values)

num_countries = []
num_items = []
for feature in feature_list:
    country_count = len(data['area'].loc[data[feature].notna()].unique())
    num_countries.append(country_count)
    item_count = len(data['item'].loc[data[feature].notna()].unique())
    num_items.append(item_count)

df = pd.DataFrame({
    'Feature': feature_list,
    'Number_of_countries': num_countries,
    'Number_of_items': num_items,
    'Percent_NA': prop_list
})

df


,Feature,Number_of_countries,Number_of_items,Percent_NA
0,fdi_ag_forest_fish_share,95,133,57.006
1,fdi_ag_forest_fish,95,133,57.006
2,govt_expenditure_on_ag_forest_fish,121,134,47.790
3,producer_price,122,133,44.676
4,agri_orientation_index_2015_usd,103,134,39.908
5,credit_to_ag_forest_fish_share_totalcredit,103,134,39.908
6,credit_to_ag_forest_fish_2015_usd,103,134,39.908
7,afs_employment_share_in_total_employment,113,134,38.795
8,total_employment_afs,116,134,37.964
9,total_govt_expenditure,124,134,32.366
